In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 256
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
        self.protein2go =  load_dict('SC_protein2go_dicts.pkl')
        self.protein2seq = load_dict('SC_protein_seqs.pkl')
        self.read_ppi()
        self.protein2onehot = {}
        self.onehot_seqs()
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
            
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

        X_go1 = np.empty((self.batch_size, self.max_golen,768))
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))

        X_go2 = np.empty((self.batch_size, self.max_golen,768))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split(',')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
            
#             X_seq1[i] =  protein_one_hot(self.protein2seq[p1], self.max_seqlen) 
#             X_seq2[i] =  protein_one_hot(self.protein2seq[p2], self.max_seqlen)
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
             
            
            
        return [  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

         
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        y = np.empty((len(list_IDs_temp)))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split(',')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
            
             
#             X_seq1[i] =  protein_one_hot(self.protein2seq[p1], self.max_seqlen) 
#             X_seq2[i] =  protein_one_hot(self.protein2seq[p2], self.max_seqlen) 
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
           
        return [  X_seq1, X_seq2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [5]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.layers.merge import concatenate
def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

def create_share_model():
    con_filters = 128
    X_input = Input(shape=(max_seq_len,20))
    # text-CNN
#     卷积池化-全连接 ---拼接-全连接  -双向GRU-全连接
    cnn = Conv1D(128, 3)(X_input)
#     cnn = BatchNormalization()(cnn)
    cnn = Activation('relu')(cnn)
     
    cnn = GlobalAveragePooling1D()(cnn)
    
    
    model = Model(X_input, cnn)
    return model


def build_siamese_model():
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    siamese_a = create_share_model()
    siamese_b = create_share_model()
    

    encoded_l = siamese_a(left_input_seq)
    encoded_r = siamese_b(right_input_seq)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
#     dense = Dense(128,activation='relu')(L1_distance)
    prediction = Dense(1,activation='sigmoid')(L1_distance)

    siamese_net = Model(inputs=[left_input_seq,right_input_seq],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return siamese_net

def build_model():
    con_filters = 128
    
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    x = inception_block(left_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(64, return_sequences=True))(left_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    left_x_seq = Concatenate()([x_a  , x_b, x_c,  x_gru_a, x_gru_b, x_gru_c])
    left_x_seq = Dense(256, activation='relu')(left_x_seq)
     
 
    x = inception_block(right_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(64, return_sequences=True))(right_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    right_x_seq= Concatenate()([x_a  , x_b, x_c, x_gru_a, x_gru_b, x_gru_c])
    right_x_seq = Dense(256, activation='relu')(right_x_seq)
    
     
    x =   Concatenate()([left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_seq, right_input_seq], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 32)     1952        input_1[0][0]                    
___________________________________________________________________________

In [7]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'SC'
for rep in range(5):
    n_splits = 1
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'SC_CV/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'SC_CV/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'SC_CV/valid'+str(rep)+'-'+str(split)

        batch_size = 256
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'SC_CV/sc_seq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('SC_seq_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)
    
    
    
    

Epoch 1/100
67/67 [==============================] - 28s 423ms/step - loss: 0.6932 - acc: 0.5286 - val_loss: 0.6801 - val_acc: 0.5555
Epoch 2/100
67/67 [==============================] - 23s 351ms/step - loss: 0.6353 - acc: 0.6374 - val_loss: 0.5992 - val_acc: 0.6773
Epoch 3/100
67/67 [==============================] - 23s 344ms/step - loss: 0.5267 - acc: 0.7361 - val_loss: 0.4591 - val_acc: 0.7910
Epoch 4/100
67/67 [==============================] - 23s 346ms/step - loss: 0.3973 - acc: 0.8218 - val_loss: 0.3821 - val_acc: 0.8340
Epoch 5/100
67/67 [==============================] - 23s 343ms/step - loss: 0.3392 - acc: 0.8501 - val_loss: 0.3101 - val_acc: 0.8791
Epoch 6/100
67/67 [==============================] - 23s 345ms/step - loss: 0.2967 - acc: 0.8765 - val_loss: 0.2795 - val_acc: 0.8897
Epoch 7/100
67/67 [==============================] - 23s 348ms/step - loss: 0.2578 - acc: 0.8933 - val_loss: 0.2916 - val_acc: 0.8909
Epoch 8/100
67/67 [==============================] - 23s 341ms

Epoch 62/100
67/67 [==============================] - 24s 352ms/step - loss: 0.1288 - acc: 0.9512 - val_loss: 0.1898 - val_acc: 0.9266
Epoch 63/100
67/67 [==============================] - 23s 343ms/step - loss: 0.1036 - acc: 0.9606 - val_loss: 0.1991 - val_acc: 0.9252
Epoch 64/100
67/67 [==============================] - 23s 347ms/step - loss: 0.0957 - acc: 0.9632 - val_loss: 0.2075 - val_acc: 0.9183
Epoch 65/100
67/67 [==============================] - 23s 342ms/step - loss: 0.0981 - acc: 0.9631 - val_loss: 0.1957 - val_acc: 0.9286
Epoch 66/100
67/67 [==============================] - 23s 341ms/step - loss: 0.1064 - acc: 0.9595 - val_loss: 0.1869 - val_acc: 0.9271
Epoch 67/100
67/67 [==============================] - 23s 339ms/step - loss: 0.0965 - acc: 0.9629 - val_loss: 0.1865 - val_acc: 0.9290
Epoch 68/100
67/67 [==============================] - 23s 343ms/step - loss: 0.1146 - acc: 0.9562 - val_loss: 0.2068 - val_acc: 0.9214
Epoch 69/100
67/67 [==============================] - 2

Epoch 31/100
67/67 [==============================] - 23s 339ms/step - loss: 0.1448 - acc: 0.9436 - val_loss: 0.2035 - val_acc: 0.9188
Epoch 32/100
67/67 [==============================] - 23s 338ms/step - loss: 0.1358 - acc: 0.9482 - val_loss: 0.2201 - val_acc: 0.9135
Epoch 33/100
67/67 [==============================] - 23s 338ms/step - loss: 0.1425 - acc: 0.9443 - val_loss: 0.1981 - val_acc: 0.9209
Epoch 34/100
67/67 [==============================] - 22s 333ms/step - loss: 0.1778 - acc: 0.9286 - val_loss: 0.2122 - val_acc: 0.9147
Epoch 35/100
67/67 [==============================] - 22s 329ms/step - loss: 0.1399 - acc: 0.9455 - val_loss: 0.1968 - val_acc: 0.9216
Epoch 36/100
67/67 [==============================] - 22s 323ms/step - loss: 0.1531 - acc: 0.9388 - val_loss: 0.1973 - val_acc: 0.9205
Epoch 37/100
67/67 [==============================] - 21s 315ms/step - loss: 0.1252 - acc: 0.9525 - val_loss: 0.1916 - val_acc: 0.9241
Epoch 38/100
67/67 [==============================] - 2

Epoch 5/100
67/67 [==============================] - 22s 331ms/step - loss: 0.3412 - acc: 0.8534 - val_loss: 0.3463 - val_acc: 0.8686
Epoch 6/100
67/67 [==============================] - 23s 338ms/step - loss: 0.2828 - acc: 0.8784 - val_loss: 0.3056 - val_acc: 0.8673
Epoch 7/100
67/67 [==============================] - 23s 337ms/step - loss: 0.2740 - acc: 0.8856 - val_loss: 0.2697 - val_acc: 0.8905
Epoch 8/100
67/67 [==============================] - 23s 340ms/step - loss: 0.2508 - acc: 0.8946 - val_loss: 0.2774 - val_acc: 0.8887
Epoch 9/100
67/67 [==============================] - 22s 334ms/step - loss: 0.2357 - acc: 0.9038 - val_loss: 0.2548 - val_acc: 0.8990
Epoch 10/100
67/67 [==============================] - 22s 331ms/step - loss: 0.2198 - acc: 0.9125 - val_loss: 0.2516 - val_acc: 0.8969
Epoch 11/100
67/67 [==============================] - 23s 336ms/step - loss: 0.2241 - acc: 0.9060 - val_loss: 0.2358 - val_acc: 0.9047
Epoch 12/100
67/67 [==============================] - 22s 33

Epoch 66/100
67/67 [==============================] - 22s 336ms/step - loss: 0.1115 - acc: 0.9577 - val_loss: 0.1971 - val_acc: 0.9232
Epoch 67/100
67/67 [==============================] - 23s 337ms/step - loss: 0.0998 - acc: 0.9625 - val_loss: 0.2439 - val_acc: 0.9103
Epoch 68/100
67/67 [==============================] - 22s 335ms/step - loss: 0.1135 - acc: 0.9568 - val_loss: 0.2044 - val_acc: 0.9212
Epoch 69/100
67/67 [==============================] - 22s 332ms/step - loss: 0.0882 - acc: 0.9671 - val_loss: 0.2129 - val_acc: 0.9165
Epoch 70/100
67/67 [==============================] - 22s 330ms/step - loss: 0.0920 - acc: 0.9655 - val_loss: 0.2115 - val_acc: 0.9160
Epoch 71/100
67/67 [==============================] - 23s 336ms/step - loss: 0.0909 - acc: 0.9661 - val_loss: 0.2222 - val_acc: 0.9113
Epoch 72/100
67/67 [==============================] - 23s 338ms/step - loss: 0.0994 - acc: 0.9633 - val_loss: 0.2365 - val_acc: 0.9100
Epoch 73/100
67/67 [==============================] - 2

Epoch 50/100
67/67 [==============================] - 22s 331ms/step - loss: 0.1455 - acc: 0.9434 - val_loss: 0.1918 - val_acc: 0.9234
Epoch 51/100
67/67 [==============================] - 22s 332ms/step - loss: 0.1335 - acc: 0.9486 - val_loss: 0.2161 - val_acc: 0.9187
Epoch 52/100
67/67 [==============================] - 22s 335ms/step - loss: 0.1174 - acc: 0.9553 - val_loss: 0.1868 - val_acc: 0.9257
Epoch 53/100
67/67 [==============================] - 22s 331ms/step - loss: 0.1040 - acc: 0.9610 - val_loss: 0.2001 - val_acc: 0.9216
Epoch 54/100
67/67 [==============================] - 22s 331ms/step - loss: 0.1081 - acc: 0.9597 - val_loss: 0.1830 - val_acc: 0.9276
Epoch 55/100
67/67 [==============================] - 22s 333ms/step - loss: 0.1051 - acc: 0.9609 - val_loss: 0.2058 - val_acc: 0.9186
Epoch 56/100
67/67 [==============================] - 22s 336ms/step - loss: 0.1144 - acc: 0.9566 - val_loss: 0.2052 - val_acc: 0.9187
Epoch 57/100
67/67 [==============================] - 2

In [8]:
import numpy as np 
results1 =   np.load( 'SC_seq_1.npz')
results2 =   np.load( 'SC_seq_0.npz')
results3 =   np.load( 'SC_seq_2.npz')
results4 =   np.load( 'SC_seq_3.npz')
results5 =   np.load( 'SC_seq_4.npz')
print ('mean AUC: %f' %  (  (np.mean( results4[ 'AUCs' ] )  +  np.mean( results5[ 'AUCs' ] )  + np.mean( results1[ 'AUCs' ] )  + np.mean(  results2[ 'AUCs' ] )  + np.mean(results3[ 'AUCs' ]))/5     ) )
print ('mean ACC: %f' %   (  ( np.mean( results4[ 'ACCs' ] )  + np.mean(  results5[ 'ACCs' ] )  +   np.mean( results1[ 'ACCs' ] )  + np.mean(  results2[ 'ACCs' ] )  + np.mean(results3[ 'ACCs' ]))/5) )
print ('mean MCC : %f' %  ( ( np.mean( results4[ 'MCCs' ] )  + np.mean(  results5[ 'MCCs' ] )  + np.mean( results1[ 'MCCs' ] )  + np.mean(  results2[ 'MCCs' ] )  + np.mean(results3[ 'MCCs' ])     )/5))
print('mean TPR:%f'%    (( np.mean( results4[ 'TPRs' ] )  + np.mean(  results5[ 'TPRs' ] )  + np.mean( results1[ 'TPRs' ] )  + np.mean(  results2[ 'TPRs' ] )  + np.mean(results3[ 'TPRs' ])     )/5))
print('mean FPR:%f'%   (  (np.mean( results4[ 'FPRs' ] )  + np.mean(  results5[ 'FPRs' ] )  + np.mean( results1[ 'FPRs' ] )  + np.mean(  results2[ 'FPRs' ] )  + np.mean(results3[ 'FPRs' ])     )/5))
print('mean Pre:%f'%    ( (np.mean( results4[ 'Precs' ] )  + np.mean(  results5[ 'Precs' ] )  + np.mean( results1[ 'Precs' ] )  + np.mean(  results2[ 'Precs' ] )  + np.mean(results3[ 'Precs' ])     )/5))
print('mean F1:%f'%    (  (np.mean( results4[ 'F1s' ] )  + np.mean(  results5[ 'F1s' ] )  +np.mean( results1[ 'F1s' ] )  + np.mean(  results2[ 'F1s' ] )  + np.mean(results3[ 'F1s' ])     )/5))

mean AUC: 0.977617
mean ACC: 0.925475
mean MCC : 0.851405
mean TPR:0.912428
mean FPR:0.061344
mean Pre:0.937520
mean F1:0.924722
